# Импорт библиотек

In [70]:
# Загружаем библиотеки
import numpy as np
import pandas as pd

# Работа с датасетом

#### Объявляем функции

In [71]:
# Чистка датасета: удаляем дубликаты, фильтруем по двухкомп. системам (так как трёхкомпонентных мало и у них не указан тип DES) и
# по плотности < 2, убираем ненужные столбцы
def dataset_clear(dataset):
    dataset = dataset.drop_duplicates(subset=['Component#1', 'Component#2', 
                                    'X#1 (molar fraction)', 'X#2 (molar fraction)', 
                                    'Temperature, K', 'Density, g/cm^3'])
    dataset = dataset[(dataset['Number of components'] == 2) & (dataset['Density, g/cm^3'] <= 2.0)]
    dataset = dataset.drop(['Number of components', 'Reference (DOI)', 'X#3 (molar fraction)', 'Component#3', 'II'], axis=1)
    return dataset

#### Подготовка датасета

In [76]:
dataset = pd.read_csv('Density.csv')

,Number of components,Type of DES,I,II,III,IV,V,Component#1,Component#2,Component#3,X#1 (molar fraction),X#2 (molar fraction),X#3 (molar fraction),"Temperature, K","Density, g/cm^3",Reference (DOI)
0,2,Ⅰ,1,0,0,0,0,Zincchloride,1-Butyl-3-methyl imidazolium chloride,NaN,0.500,0.500,NaN,300.20,1.10250,10.1016/j.cclet.2013.09.004
1,2,Ⅰ,1,0,0,0,0,Zincchloride,1-Butyl-3-methylimidazolium chloride,NaN,0.500,0.500,NaN,303.15,1.50913,10.1016/j.jobe.2021.103056
2,2,Ⅰ,1,0,0,0,0,Zincchloride,1-Butyl-3-methylimidazolium chloride,NaN,0.333,0.667,NaN,303.15,1.34292,10.1016/j.jobe.2021.103056
3,2,Ⅰ,1,0,0,0,0,Iron(3+) Trichloride,Tetrabutylphosphonium bromide,NaN,0.333,0.667,NaN,293.00,1.19500,10.1016/j.cep.2015.04.001
4,2,Ⅰ,1,0,0,0,0,Iron(3+) Trichloride,Tetrabutylphosphonium bromide,NaN,0.333,0.667,NaN,303.00,1.18800,10.1016/j.cep.2015.04.001


In [79]:
# Чистим датасет
pre_dataset = dataset_clear(dataset)